## Main()
- Ler toda a pasta, pegar a string de user, dezipar, ler os arquivos .csv e passar para as dfs
- Tornar esse código main

In [5]:
import os
import zipfile
import shutil

def unzip_and_delete(directory):
    # Listar arquivos no diretório
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        # Verificar se é um arquivo e se tem extensão .zip
        if os.path.isfile(item_path) and item.endswith('.zip'):
            print(f'Encontrado arquivo zip: {item}')
            # Extrair o arquivo zip
            with zipfile.ZipFile(item_path, 'r') as zip_ref:
                extract_path = os.path.join(directory, os.path.splitext(item)[0])
                zip_ref.extractall(extract_path)
                print(f'Extraído para: {extract_path}')
                # Remover a pasta extraída
                shutil.rmtree(extract_path)
                print(f'Pasta removida: {extract_path}')

# Uso do exemplo
starting_directory = r'../data\raw'
unzip_and_delete(starting_directory)

Root: ..\data\raw
  Directory: letterboxd-gbmonteiro-2024-05-14-22-51-utc
  File: .gitkeep

Root: ..\data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc
  Directory: deleted
  Directory: likes
  Directory: orphaned
  File: comments.csv
  File: diary.csv
  File: profile.csv
  File: ratings.csv
  File: reviews.csv
  File: watched.csv
  File: watchlist.csv

Root: ..\data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\deleted
  File: comments.csv
  File: diary.csv
  File: reviews.csv

Root: ..\data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\likes
  File: films.csv
  File: lists.csv
  File: reviews.csv

Root: ..\data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\orphaned
  File: comments.csv
  File: diary.csv
  File: reviews.csv



In [ ]:
import pandas as pd

# Load the CSV file
ratings = pd.read_csv(r'../data/raw/letterboxd-gbmonteiro-2024-05-14-22-51-utc/ratings.csv')

ltbxd_scp = pd.read_csv(r'../data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\watched.csv')

## PG connection - Pegando dados do PG

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from Pgconnection import ReturningDF


# Define the query to fetch data
query = 'SELECT * FROM public.moviesdb'

# Call the ReturningDF function
ltbd_pg = ReturningDF(query)

# Check the result
if ltbd_pg is not None:
    print(ltbd_pg.head())
else:
    print("Failed to retrieve data.")
    

         Date           Name  Year        Letterboxd_URI  \
0  2024-01-15         Barbie  2023  https://boxd.it/bCLK   
1  2024-01-16   Come and See  1985  https://boxd.it/1w9q   
2  2024-01-16  The Sacrifice  1986  https://boxd.it/1xg8   
3  2024-01-16        The Fly  1986  https://boxd.it/1YIe   
4  2024-01-16  Boogie Nights  1997  https://boxd.it/23um   

               Director                   Genres  \
0          Greta Gerwig        Comedy; Adventure   
1           Elem Klimov               War; Drama   
2      Andrei Tarkovsky                    Drama   
3      David Cronenberg  Science Fiction; Horror   
4  Paul Thomas Anderson                    Drama   

                                              Themes  \
0  Humanity and the world around us; Crude humor ...   
1  War and historical adventure; Intense violence...   
2  Faith and religion; Humanity and the world aro...   
3  Horror, the undead and monster classics; Inten...   
4  Moving relationship stories; Intense violen

c:\Users\guibe\anaconda3\envs\letterboxd_projeto\letterboxd_recommendation\notebooks\Pgconnection.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Datascraper - Adicionando os dados

In [2]:
ltbxd_scp = pd.read_csv(r'../data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\watched.csv')

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from Pgconnection import AddRowToTable

ltbxd_scp.rename(columns={'Letterboxd URI': 'Letterboxd_URI'}, inplace=True)

# Assuming ltbxd_wtchd is your merged DataFrame
for index, row in ltbxd_scp.iterrows():
    title = row['Name']
    
    # Check if the movie already exists in the PostgreSQL DataFrame
    if title in ltbd_pg['Name'].values:
        print(f"Skipping scraping for {title}. Data already exists in PostgreSQL.")
        continue
    
    url = row['Letterboxd_URI']
    
    # Add user-agent headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    # Sending a GET request to the URL
    response = requests.get(url, headers=headers)
    
    # Checking if the request was successful (status code 200)
    if response.status_code == 200:
        # Parsing the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Now you can work with the parsed HTML content
        # For example, you can extract specific information from the webpage
        
        # For demonstration, let's print the title of the webpage
        print(f"Title of webpage {index}: {soup.title.string}")
        
        # Find the span with class 'directorlist'
        director_span = soup.find('span', class_='directorlist')

        # Extract the director's name from the span
        director_name = director_span.text.strip()
        
        # Extract genres from the specific section
        genre_section = soup.find('div', id='tab-genres')
        if genre_section:
            genre_tags = genre_section.find_all('a', class_='text-slug')
            genres = [tag.text for tag in genre_tags if '/films/genre/' in tag['href']]
        else:
            genres = ["Genres not found"]
        
        # Extract themes from the specific section
        if genre_section:
            theme_tags = genre_section.find_all('a', class_='text-slug')
            themes = [tag.text for tag in theme_tags if '/films/theme/' in tag['href'] or '/films/mini-theme/' in tag['href']]
        else:
            themes = ["Themes not found"]
        
        # Extract actors from the specific section
        actor_section = soup.find('div', id='tab-cast')
        if actor_section:
            actor_tags = actor_section.find_all('a', class_='text-slug')
            actors = [tag.text for tag in actor_tags]
        else:
            actors = ["Actors not found"]

             
        # Update the DataFrame with the director, genres, and themes
        ltbxd_scp.at[index, 'Director'] = director_name
        ltbxd_scp.at[index, 'Genres'] = '; '.join(genres)
        ltbxd_scp.at[index, 'Themes'] = '; '.join(themes)
        ltbxd_scp.at[index, 'Actors'] = '; '.join(actors)
        
        # Pass row as a dictionary to AddRowToTable function
        movie_data = tuple(ltbxd_scp.iloc[index])

        # Convert numpy.int64 types to native Python int types
        movie_data = tuple(int(item) if isinstance(item, np.int64) else item for item in movie_data)

        # Call AddRowToTable function with the table name and movie data
        AddRowToTable(movie_data)
    else:
        print(f"Failed to fetch data from URL {url} with status code {response.status_code}")


Skipping scraping for Barbie. Data already exists in PostgreSQL.
Skipping scraping for Come and See. Data already exists in PostgreSQL.
Skipping scraping for The Sacrifice. Data already exists in PostgreSQL.
Skipping scraping for The Fly. Data already exists in PostgreSQL.
Skipping scraping for Boogie Nights. Data already exists in PostgreSQL.
Skipping scraping for La Haine. Data already exists in PostgreSQL.
Skipping scraping for Alien. Data already exists in PostgreSQL.
Skipping scraping for Volver. Data already exists in PostgreSQL.
Skipping scraping for São Paulo, Sociedade Anônima. Data already exists in PostgreSQL.
Skipping scraping for Ready or Not. Data already exists in PostgreSQL.
Skipping scraping for The Menu. Data already exists in PostgreSQL.
Skipping scraping for Oppenheimer. Data already exists in PostgreSQL.
Skipping scraping for Elis & Tom: It Had to be You. Data already exists in PostgreSQL.
Skipping scraping for The Given Word. Data already exists in PostgreSQL.
Ski

## PG connection - Adicionando dados de users

In [ ]:
import pandas as pd
# Load the CSV file
ratings = pd.read_csv(r'../data/raw/letterboxd-gbmonteiro-2024-05-14-22-51-utc/ratings.csv')

In [2]:
from Pgconnection import EnteringTable

# Add a new column 'User' with the value 'gbmonteiro' for all rows
ratings['User'] = 'gbmonteiro'

# Select only the required columns: 'User', 'Date', 'Rating'
ratings = ratings[['User', 'Date', 'Rating']]

# Insert the DataFrame into the 'public.ratings' table
EnteringTable("ratings", ratings)